process_NLDAS_extremes.ipynb  
Casey D. Burleyson   
Pacific Northwest National Laboratory  
17-Jan 2022
  
This script takes arbitrary lists of NLDAS output files and computes the minimum and maximum values of a series of meteorological variables. These min/max values can be used as a first pass to identify outliers in the IM3 WRF simulations. All times are in UTC. The code as written computes minimum and maximum values for the following variables:
1. 10-m wind speed; WSPD; m/s
2. 2-m temperature; T2; K
3. Surface pressure; PS; Pa
4. 2-m specific humidity; Q2; kg/kg
5. Downwelling longwave radiation at the surface; GLW; W/m^2
6. Downwelling shortwave radiation at the surface; SWDOWN; W/m^2

In [1]:
# Import all of the required libraries and packages:
import os
import glob
import pandas as pd
import xarray as xr
import numpy as np

# Set the data input and output directories:
data_input_dir = '/Volumes/LaCie/NLDAS/Raw_Data/'
data_output_dir = '/Volumes/LaCie/NLDAS/'

In [2]:
# Define a function to read in a single NLDAS file and extract the minimum and maximum value of several meteorological variables:
def process_single_hour_min_max_values(filename):
    # Read in the NLDAS file using xarray:
    nldas = xr.open_dataset(filename)

    # Create a new dataframe to output the results:
    output_df = pd.DataFrame()

    # Store the time variable as a pandas datetime:
    output_df.at[0,'Time'] = pd.to_datetime(nldas.time).item()

    # Compute and store the maximum and minimum wind speeds
    output_df.at[0,'WSPD_Min'] = (np.sqrt(np.square(nldas.UGRD) + np.square(nldas.VGRD))).min().item() # Minimum 10-m wind speed in m/s
    output_df.at[0,'WSPD_Max'] = (np.sqrt(np.square(nldas.UGRD) + np.square(nldas.VGRD))).max().item() # Maximum 10-m wind speed in m/s

    # Store the maximum and minimum temperature, surface pressure, specific humidity, and downwelling and long- and shortwave radiation:
    output_df.at[0,'T2_Min'] = nldas.TMP.min().item() # Minimum 2-m temperature in K
    output_df.at[0,'T2_Max'] = nldas.TMP.max().item() # Maximum 2-m temperature in K
    output_df.at[0,'PS_Min'] = nldas.PRES.min().item() # Minimum surface pressure in Pa
    output_df.at[0,'PS_Max'] = nldas.PRES.max().item() # Maximum surface pressure in Pa
    output_df.at[0,'Q2_Min'] = nldas.SPFH.min().item() # Minimum 2-m specific humidity in kg/kg
    output_df.at[0,'Q2_Max'] = nldas.SPFH.max().item() # Maximum 2-m specific humidity in kg/kg
    output_df.at[0,'GLW_Min'] = nldas.DLWRF.min().item() # Minimum downwelling longwave radiation at the surface in W/m^2
    output_df.at[0,'GLW_Max'] = nldas.DLWRF.max().item() # Maximum downwelling longwave radiation at the surface in W/m^2
    output_df.at[0,'SWDOWN_Min'] = nldas.DSWRF.min().item() # Minimum downwelling shortwave radiation at the surface in W/m^2
    output_df.at[0,'SWDOWN_Max'] = nldas.DSWRF.max().item() # Maximum downwelling shortwave radiation at the surface in W/m^2

    # Round off the decimal places for all variables:
    output_df['WSPD_Min'] = output_df['WSPD_Min'].round(2)
    output_df['WSPD_Max'] = output_df['WSPD_Max'].round(2)
    output_df['T2_Min'] = output_df['T2_Min'].round(2)
    output_df['T2_Max'] = output_df['T2_Max'].round(2)
    output_df['PS_Min'] = output_df['PS_Min'].round(2)
    output_df['PS_Max'] = output_df['PS_Max'].round(2)
    output_df['Q2_Min'] = output_df['Q2_Min'].round(6)
    output_df['Q2_Max'] = output_df['Q2_Max'].round(6)
    output_df['GLW_Min'] = output_df['GLW_Min'].round(2)
    output_df['GLW_Max'] = output_df['GLW_Max'].round(2)
    output_df['SWDOWN_Min'] = output_df['SWDOWN_Min'].round(2)
    output_df['SWDOWN_Max'] = output_df['SWDOWN_Max'].round(2)
    
    return output_df

In [3]:
list_of_files = sorted(glob.glob(os.path.join(data_input_dir + 'NLDAS_FORA0125_H.A*.002.grb.SUB.nc4')))

# Loop over the states and interpolate their loads to an annual time step:
for file in range(len(list_of_files)):
    # If this is the first step in the loop then create a new output dataframe to store the results else just append the results to the existing output dataframe:
    if file == 0:
       hourly_min_max_df = process_single_hour_min_max_values(list_of_files[file])
    else:
       hourly_min_max_df = hourly_min_max_df.append(process_single_hour_min_max_values(list_of_files[file]))

# Create strings of the starting and ending times and generate the .csv output file name:
start_time = str(hourly_min_max_df['Time'].min()).replace(" ","_").replace("-","_").replace(":","_").replace("_00_00","")
end_time = str(hourly_min_max_df['Time'].max()).replace(" ","_").replace("-","_").replace(":","_").replace("_00_00","")
hourly_csv_output_filename = os.path.join(data_output_dir,'NLDAS_Hourly_Min_Max_Values_' + start_time + '_UTC_to_' + end_time + '_UTC.csv')

# Save the hourly output dataframe to a .csv file:
hourly_min_max_df.to_csv(hourly_csv_output_filename, sep=',', index=False)

# Display the head of the hourly output dataframe:
hourly_min_max_df.head(10)

,Time,WSPD_Min,WSPD_Max,T2_Min,T2_Max,PS_Min,PS_Max,Q2_Min,Q2_Max,GLW_Min,GLW_Max,SWDOWN_Min,SWDOWN_Max
0,2010-01-01 00:00:00,0.01,18.79,246.26,300.64,64847.87,103447.55,0.000353,0.015923,118.70,420.12,0.0,156.89
0,2010-01-01 01:00:00,0.01,17.65,244.96,297.71,64822.44,103454.12,0.000317,0.015988,118.68,420.12,0.0,4.02
0,2010-01-01 02:00:00,0.01,16.58,243.66,297.28,64797.00,103461.32,0.000281,0.016062,118.66,420.12,0.0,0.00
0,2010-01-01 03:00:00,0.01,16.38,242.36,297.28,64771.56,103483.24,0.000244,0.016291,113.85,426.22,0.0,0.00
0,2010-01-01 04:00:00,0.01,17.05,241.84,297.26,64833.84,103613.36,0.000231,0.016205,113.84,426.22,0.0,0.00
0,2010-01-01 05:00:00,0.02,18.59,241.32,297.25,64886.65,103747.45,0.000218,0.016286,113.83,426.22,0.0,0.00
0,2010-01-01 06:00:00,0.03,20.23,240.76,297.23,64939.46,103887.94,0.000204,0.016366,111.90,425.55,0.0,0.00
0,2010-01-01 07:00:00,0.01,19.71,240.38,297.13,64954.73,103764.33,0.000194,0.016299,111.89,425.55,0.0,0.00
0,2010-01-01 08:00:00,0.01,19.19,239.57,297.03,64936.01,103640.65,0.000182,0.016230,111.88,425.55,0.0,0.00
0,2010-01-01 09:00:00,0.01,18.67,238.57,296.93,64917.23,103516.91,0.000162,0.016163,112.20,428.10,0.0,0.00


In [4]:
# Create a new dataframe to output the global max/min results:
global_min_max_df = pd.DataFrame()

# Store the minimum and maximum values over the whole time period processed:
global_min_max_df.at[0,'Start_Time'] = start_time
global_min_max_df.at[0,'End_Time'] = end_time
global_min_max_df.at[0,'WSPD_Min'] = hourly_min_max_df['WSPD_Min'].min()
global_min_max_df.at[0,'WSPD_Max'] = hourly_min_max_df['WSPD_Max'].max()
global_min_max_df.at[0,'T2_Min'] = hourly_min_max_df['T2_Min'].min()
global_min_max_df.at[0,'T2_Max'] = hourly_min_max_df['T2_Max'].max()
global_min_max_df.at[0,'PS_Min'] = hourly_min_max_df['PS_Min'].min()
global_min_max_df.at[0,'PS_Max'] = hourly_min_max_df['PS_Max'].max()
global_min_max_df.at[0,'Q2_Min'] = hourly_min_max_df['Q2_Min'].min()
global_min_max_df.at[0,'Q2_Max'] = hourly_min_max_df['Q2_Max'].max()
global_min_max_df.at[0,'GLW_Min'] = hourly_min_max_df['GLW_Min'].min()
global_min_max_df.at[0,'GLW_Max'] = hourly_min_max_df['GLW_Max'].max()
global_min_max_df.at[0,'SWDOWN_Min'] = hourly_min_max_df['SWDOWN_Min'].min()
global_min_max_df.at[0,'SWDOWN_Max'] = hourly_min_max_df['SWDOWN_Max'].max()

# Generate the .csv output file name:
global_csv_output_filename = os.path.join(data_output_dir,'NLDAS_Global_Min_Max_Values_' + start_time + '_UTC_to_' + end_time + '_UTC.csv')

# Save the hourly output dataframe to a .csv file:
global_min_max_df.to_csv(global_csv_output_filename, sep=',', index=False)

# Display the output dataframe:
global_min_max_df

,Start_Time,End_Time,WSPD_Min,WSPD_Max,T2_Min,T2_Max,PS_Min,PS_Max,Q2_Min,Q2_Max,GLW_Min,GLW_Max,SWDOWN_Min,SWDOWN_Max
0,2010_01_01_00,2010_12_31_23,0.0,27.85,233.19,322.34,62144.47,104247.57,0.0,0.028547,93.19,544.93,0.0,1363.76
